In [2]:
import sqlite3
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import missingno as msno
from matplotlib.pyplot import figure
import seaborn as sns
from scipy.stats import norm, skew 
from scipy import stats
from datetime import datetime
import scipy.stats
from sklearn.model_selection import train_test_split, KFold, cross_validate,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score,f1_score, mean_squared_error,mean_absolute_error
import statsmodels.api as sm
import warnings
warnings.filterwarnings(action="ignore")
from tqdm import tqdm

from statsmodels.tsa.ar_model import AutoReg,AR
import json
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler
import random

from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA

In [30]:
df=pd.read_csv('new_data_climate_pop.csv')
df['date']=pd.to_datetime(df['date'])
df1=(df.set_index('date',drop=True)).sort_values('state')
states=df1['state'].unique()
df1.sort_values(['state','date'],inplace=True)
lag_value=1
df1['school_mode'] = df1['school_mode'].round().astype('float')
df1['elderly_mode'] = df1['elderly_mode'].round().astype('float')
mapping = {0: 'no_policy', 1: 'low_policy', 2: 'med_policy', 3: 'high_policy'}
df1['elderly_mode'] = df1['elderly_mode'].astype(int).map(mapping)
df1['school_mode'] = df1['school_mode'].astype(int).map(mapping)

state_dfs = []

# List of unique states in your DataFrame
unique_states = df1['state'].unique()

# Loop through each state
for state in unique_states:
    # Filter the data for the current state
    state_df = df1[df1['state'] == state].copy()

    # Apply your operations to the state-specific DataFrame
    scaler = StandardScaler()
    state_df['diff_ilip'] = state_df['ILIp'].diff()
    state_df['smoothed_ILIp'] = state_df['diff_ilip'].rolling(window=7).mean()
    state_df = state_df.dropna()
    state_df['school_mode_lag'] = state_df['school_mode'].shift(4)
    state_df['elderly_mode_lag'] = state_df['elderly_mode'].shift(4)
    state_df['school_mode_lag_temp'] = state_df['school_mode_lag']
    state_df['elderly_mode_lag_temp'] = state_df['elderly_mode_lag']
    #state_df[['mask','pca_1','ILIp','resid_av7','unweighted_ili']] = scaler.fit_transform(state_df[['mask','pca_1','ILIp','resid_av7','unweighted_ili']])
    state_df['ILIp_lag'] = state_df['ILIp'].shift(1)
    state_df.sort_values(['state','date'], inplace=True, ascending=True)
    state_df = pd.get_dummies(state_df, columns=['school_mode_lag'], prefix='school_mode')
    state_df = pd.get_dummies(state_df, columns=['elderly_mode_lag'], prefix='elderly_mode')
    state_df['mask_lag'] = state_df['mask'].shift(4)
    state_df['resid_av7_lag'] = state_df['resid_av7'].shift(4)
    state_df['pca_1_lag'] = state_df['pca_1'].shift(4)

    state_df = state_df[:'2022-06-01'].dropna()
    
    # Append the state-specific DataFrame to the combined DataFrame
    state_dfs.append(state_df)

# Print the shape of the combined DataFrame and display the first few rows
combined_df = pd.concat(state_dfs[:])
combined_df1=combined_df.sort_values(['date','state'])
#state_mapping = {state: index for index, state in enumerate(unique_states, start=1)}
#combined_df1['state'] = combined_df1['state'].map(state_mapping)
combined_df1.reset_index(inplace=True)

combined_df1.head()

,date,state,elderly_mode,school_mode,unweighted_ili,mask,rr_av7,gp_av7,transit_av7,work_av7,...,school_mode_high_policy,school_mode_low_policy,school_mode_med_policy,elderly_mode_low_policy,elderly_mode_med_policy,mask_lag,resid_av7_lag,pca_1_lag,school_mode_no_policy,elderly_mode_high_policy
0,2020-05-17,AK,med_policy,high_policy,0.472335,0.4375,-1.285714,9.285714,-27.000000,-25.571429,...,1.0,0,0.0,0.0,1.0,0.4015,14.142857,-40.983298,NaN,NaN
1,2020-05-17,AL,high_policy,high_policy,1.033770,0.4269,-10.142857,3.857143,-9.285714,-26.428571,...,1.0,0,0.0,0.0,0.0,0.3867,14.000000,-26.770484,0.0,1.0
2,2020-05-17,AR,high_policy,high_policy,0.739881,0.5468,-7.142857,6.857143,-10.428571,-25.428571,...,1.0,0,0.0,0.0,0.0,0.4587,11.285714,-16.105589,0.0,1.0
3,2020-05-17,AZ,low_policy,high_policy,1.224650,0.3952,-22.428571,-6.571429,-30.857143,-34.571429,...,1.0,0,0.0,1.0,0.0,0.3666,15.285714,-41.654771,0.0,NaN
4,2020-05-17,CA,high_policy,high_policy,1.060590,0.6258,-44.285714,-9.714286,-45.428571,-40.285714,...,1.0,0,0.0,NaN,0.0,0.6229,20.285714,-63.949901,NaN,1.0


In [31]:
%load_ext sql
conn = sqlite3.connect('influenza.db')  # Change to your desired database name
combined_df1.to_sql('flue_data', conn, index=False, if_exists='replace')  # Change 'your_table_name' to your desired table name
%sql sqlite:///influenza.db

# Execute an SQL query using magic command
%sql SELECT * FROM data limit 5;

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


state,elderly_mode,school_mode,unweighted_ili,mask,rr_av7,gp_av7,transit_av7,work_av7,resid_av7,ILIp,climate,spec.hum,Population,pca_1,diff_ilip,smoothed_ILIp,school_mode_lag_temp,elderly_mode_lag_temp,ILIp_lag,school_mode_high_policy,school_mode_low_policy,school_mode_med_policy,elderly_mode_low_policy,elderly_mode_med_policy,mask_lag,resid_av7_lag,pca_1_lag,school_mode_no_policy,elderly_mode_high_policy
AK,med_policy,high_policy,0.4723350000000001,0.4375,-1.28571428571429,9.28571428571428,-27.0,-25.5714285714286,7.28571428571428,0.0,-0.2463648821881894,0.0040759416579345,"732,923",-4.93028610122877,-1.6109945808e-05,0.0,high_policy,med_policy,1.6109945808e-05,1.0,0,0.0,0.0,1.0,0.4015,14.142857142857098,-40.98329817045983,None,None
AL,high_policy,high_policy,1.03377,0.4269,-10.1428571428571,3.85714285714286,-9.28571428571428,-26.4285714285714,8.71428571428571,4.7553420000000006e-05,-0.8822965328127099,0.0115000351684742,"5,031,362",3.2918602641867123,4.7553420000000006e-05,-1.865539e-05,high_policy,high_policy,0.0,1.0,0,0.0,0.0,0.0,0.3867,14.0,-26.770483838169845,0.0,1.0
AR,high_policy,high_policy,0.739881,0.5468,-7.142857142857139,6.857142857142861,-10.4285714285714,-25.4285714285714,7.71428571428571,0.0,-0.5962060776621578,0.0106380354588701,"3,014,195",4.977530714091056,0.0,-3.960759814285714e-05,high_policy,high_policy,0.0,1.0,0,0.0,0.0,0.0,0.4587,11.2857142857143,-16.105589138526046,0.0,1.0
AZ,low_policy,high_policy,1.22465,0.3952,-22.4285714285714,-6.571428571428571,-30.857142857142897,-34.5714285714286,11.2857142857143,0.0,-0.0459740126919482,0.0036615353897912,"7,179,943",-24.813812043978743,0.0,0.0,high_policy,low_policy,0.0,1.0,0,0.0,1.0,0.0,0.3666,15.2857142857143,-41.654771087176016,0.0,None
CA,high_policy,high_policy,1.06059,0.6258,-44.28571428571429,-9.714285714285714,-45.428571428571395,-40.2857142857143,16.428571428571402,0.0,-0.0315367302535405,0.0070402628020556,"39,501,653",-49.1224769770192,-2.83395e-05,-2.8558860000000003e-05,high_policy,high_policy,2.83395e-05,1.0,0,0.0,None,0.0,0.6229,20.2857142857143,-63.9499006007164,None,1.0


In [35]:
#sql queries
%sql SELECT COUNT(*) AS total_records,MIN(ILIp) AS min_ILIp,MAX(ILIp) AS max_ILIp,AVG(ILIp) AS avg_ILIp FROM data;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


total_records,min_ILIp,max_ILIp,avg_ILIp
5169,0.0,0.01171253664,0.00034926312869083847


In [54]:
#Elderly and School Policies Analysis:
%sql SELECT elderly_mode,ROUND(AVG(ILIp),7) AS flue_transmission  FROM data GROUP BY elderly_mode;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


elderly_mode,flue_transmission
high_policy,3.98e-05
low_policy,0.0005715
med_policy,0.000325


In [60]:
%sql SELECT school_mode,ROUND(AVG(ILIp),7) AS flue_transmission  FROM data GROUP BY school_mode;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


school_mode,flue_transmission
high_policy,1.6e-05
low_policy,0.0004867
med_policy,4.91e-05
no_policy,0.0010443


In [61]:
%sql SELECT elderly_mode,ROUND(AVG(mask),7) AS flue_transmission  FROM data GROUP BY elderly_mode;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


elderly_mode,flue_transmission
high_policy,0.6484585
low_policy,0.3675459
med_policy,0.4769367


In [62]:
%sql SELECT school_mode,ROUND(AVG(mask),7) AS flue_transmission  FROM data GROUP BY school_mode;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


school_mode,flue_transmission
high_policy,0.5979209
low_policy,0.3693746
med_policy,0.643692
no_policy,0.2408357


In [48]:
#Geographical Analysis:

%sql SELECT state,ROUND(AVG(ILIp),7) AS flue_transmission FROM data GROUP BY state ORDER BY flue_transmission DESC limit 10;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


state,flue_transmission
FL,0.0023267
MS,0.0008917
NM,0.0008641
NE,0.000832
TX,0.0007382
TN,0.000643
WY,0.000599
AR,0.0005685
GA,0.0005675
OH,0.0005568


In [57]:
%sql SELECT state,ROUND(AVG(mask),7) AS flue_transmission FROM data GROUP BY state ORDER BY flue_transmission DESC limit 10;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


state,flue_transmission
HI,0.6676029
CA,0.6159562
MD,0.6008916
NM,0.596271
NY,0.5948056
CT,0.5815276
WA,0.5779095
IL,0.574121
NJ,0.5715819
NV,0.5639971


In [59]:
%sql SELECT state,ROUND(AVG(pca_1),7) AS flue_transmission FROM data GROUP BY state ORDER BY flue_transmission DESC limit 10;


   sqlite:///Walmart.db
 * sqlite:///influenza.db
Done.


state,flue_transmission
MT,41.0239728
SD,40.9868902
WY,33.780347
MS,29.0593747
ID,28.6076736
AR,22.4871109
AK,22.1399858
ME,21.8185292
OK,21.3444596
SC,20.6095372
